In [1]:
!mkdir data/ 
!wget https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar -O data/ILSVRC2012_img_val.tar
!tar -xf data/ILSVRC2012_img_val.tar -C data/
!wget https://raw.githubusercontent.com/Evolving-AI-Lab/ppgn/master/misc/map_clsloc.txt
!wget https://raw.githubusercontent.com/yqihao/PTMValidations/main/imagenet_classes.txt
!wget https://raw.githubusercontent.com/yqihao/PTMValidations/main/ILSVRC2012_validation_ground_truth.txt

--2022-05-02 17:32:30--  https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar
Resolving image-net.org (image-net.org)... 171.64.68.16
Connecting to image-net.org (image-net.org)|171.64.68.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6744924160 (6.3G) [application/x-tar]
Saving to: ‘data/ILSVRC2012_img_val.tar’

data/ILSVRC2012_img 100%[===================>]   6.28G  11.5MB/s    in 12m 41s 

2022-05-02 17:45:11 (8.45 MB/s) - ‘data/ILSVRC2012_img_val.tar’ saved [6744924160/6744924160]

--2022-05-02 17:45:42--  https://raw.githubusercontent.com/Evolving-AI-Lab/ppgn/master/misc/map_clsloc.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24366 (24K) [text/plain]
Saving to: ‘map_clsloc.txt’

map_clsloc.txt      100%[====

In [2]:
import tensorflow as tf
import numpy as np

model = tf.keras.applications.EfficientNetB3(weights='imagenet')

mapping_file = open('map_clsloc.txt',mode='r')

validation_file = open('ILSVRC2012_validation_ground_truth.txt',mode='r')

validation_list = validation_file.readlines()
mapping_list_from = mapping_file.readlines()
mapping_list_to = []
for i in range(len(mapping_list_from)):
  mapping_list_to.append(mapping_list_from[i].split(" ")[1])
  mapping_list_from[i] = mapping_list_from[i].split(" ")[0]

for i in range(len(validation_list)):
  validation_list[i] = validation_list[i].split('\n')[0]

50110464/50095040 [==============================] - 1s 0us/step


In [4]:
correct = 0
top_5 = 0

it = 50000

for i in range(it):
  img_path = f'data/ILSVRC2012_val_{(i+1):08d}.JPEG'
  img = tf.keras.preprocessing.image.load_img(img_path, target_size=(300, 300))
  x = tf.keras.preprocessing.image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = tf.keras.applications.efficientnet.preprocess_input(x)
  preds = model.predict(x)

  id = tf.keras.applications.imagenet_utils.decode_predictions(preds, top=1)[0][0][0]
  index = mapping_list_from.index(id)

  if mapping_list_to[index] == validation_list[i]:
    correct = correct + 1
    top_5 = top_5 + 1
  else:
    for ii in range(1,5):
      id = tf.keras.applications.imagenet_utils.decode_predictions(preds, top=5)[0][ii][0]
      index = mapping_list_from.index(id)
      if mapping_list_to[index] == validation_list[i]:
        top_5 = top_5 + 1
        break

  if i!=0 and i%500==0:
    print(str(int(i/500))+'%, Top 1:' + str(correct/i) + ' Top 5:' + str(top_5/i))

accuracy = correct/it*100
top_5_accuracy = top_5/it*100
print('Top 1 Accuracy:'+str(accuracy)+'%')
print('Top 5 Accuracy:'+str(top_5_accuracy)+'%')

49152/35363 [=========================================] - 0s 0us/step
1%, Top 1:0.786 Top 5:0.942
2%, Top 1:0.784 Top 5:0.943
3%, Top 1:0.802 Top 5:0.9433333333333334
4%, Top 1:0.799 Top 5:0.9385
5%, Top 1:0.8 Top 5:0.9412
6%, Top 1:0.8006666666666666 Top 5:0.9416666666666667
7%, Top 1:0.7971428571428572 Top 5:0.9451428571428572
8%, Top 1:0.793 Top 5:0.944
9%, Top 1:0.7971111111111111 Top 5:0.9448888888888889
10%, Top 1:0.797 Top 5:0.9444
11%, Top 1:0.798 Top 5:0.9441818181818182
12%, Top 1:0.798 Top 5:0.9446666666666667
13%, Top 1:0.798923076923077 Top 5:0.9452307692307692
14%, Top 1:0.7991428571428572 Top 5:0.9465714285714286
15%, Top 1:0.7998666666666666 Top 5:0.9461333333333334
16%, Top 1:0.799625 Top 5:0.946625
17%, Top 1:0.8001176470588235 Top 5:0.9469411764705883
18%, Top 1:0.7974444444444444 Top 5:0.9471111111111111
19%, Top 1:0.7977894736842105 Top 5:0.9475789473684211
20%, Top 1:0.7971 Top 5:0.9468
21%, Top 1:0.7963809523809524 Top 5:0.9471428571428572
22%, Top 1:0.7952727272